### **1. 많이 본 뉴스 데이터 수집**

In [ ]:
#코드 13-1
import sys

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
#코드 13-2
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service = webdriver_service, options=options)
url = 'http://www.korea.kr/news/top50List.do'
driver.get(url)

In [ ]:
#코드 13-3
columns = ['순위', '기사 제목', '분류', '기사 작성일']
rank = []
title = []
journal = []
date = []
for i in range(1, 51):
rank.append(i)
title.append(driver.find_element(By.CSS_SELECTOR, '#container > div > div > div >\
div.news_top50 > ul > li:nth-child(' + str(i) + ') > a > span.text >\
strong').text)
journal.append(driver.find_element(By.CSS_SELECTOR, '#container > div > div >\
div > div.news_top50 > ul > li:nth-child(' + str(i) +') > a >\
span.text > span.date > span:nth-child(1)').text)
date.append(driver.find_element(By.CSS_SELECTOR, '#container > div > div > div >\
div.news_top50 > ul > li:nth-child(' + str(i) +') > a > span.text >\
span.date > span:nth-child(2)').text)

In [ ]:
#코드 13-4
import pandas as pd
df = pd.DataFrame({columns[0]: rank,
                   columns[1]: title,
                   columns[2]: journal,
                   columns[3]: date}, columns=columns)
df.head()

In [ ]:
#코드 13-5
df.to_csv('top50_news.csv', index=False)

### **2. 많이 본 뉴스 워드클라우드**

In [ ]:
#코드 13-6
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#코드 13-7
import csv

#CSV 파일 읽어오기
f = open('top50_news.csv', 'r')
rdr = csv.reader(f)
f.close

title = ''
for line in rdr:
  title = title + ' ' + line[1]
print(title)

In [ ]:
#코드 13-8
#인덱스 0~5를 제외하고 6부터 슬라이싱하기
title = title[6:]
print(title)

In [ ]:
#코드 13-9
!pip install konlpy

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
import numpy as np

In [ ]:
#코드 13-10
#문자열 분석하기
okt = Okt()
nouns = okt.nouns(title)
words = [n for n in nouns if len(n) > 1]
c = Counter(words)

#워드클라우드 생성하기
wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',\
               scale=2.0, colormap='Spectral')
gen = wc.generate_from_frequencies(c)
plt.figure()
plt.imshow(gen)